In [3]:
import pandas as pd
import cbsodata 

toc = pd.DataFrame(cbsodata.get_table_list())

In [45]:
class CBSDataDownloader(): 
    def __init__(self, start_year=2012, end_year=2024): 
        self.start_year = start_year
        self.end_year = end_year
        self.get_table_list()
        self.identifier_dict = {}

    def get_table_list(self): 
        toc = pd.DataFrame(cbsodata.get_table_list())
        self.table_list = toc[toc['Title'].str.contains('kerncijfers wijken en buurten', case=False)]

    def get_identifier(self, year): 
        table_list = self.table_list
        return table_list[table_list['Title'].str.contains(str(year))]['Identifier'].values[0]

    def download_data(self, year): 
        df = pd.DataFrame(cbsodata.get_data(self.identifier_dict[year], select=[
                # 'WijkenEnBuurten_0', 
                'Gemeentenaam_1', 'SoortRegio_2', 'Codering_3', 
                'AantalInwoners_5', 'HuishoudensTotaal_28', 'GemiddeldeHuishoudensgrootte_32', 
                'GemiddeldElektriciteitsverbruikTotaal_47', 'GemiddeldAardgasverbruikTotaal_55', 
                'PercentageWoningenMetStadsverwarming_63'
                ]))
        df.to_csv(f'data/raw/cbs/wijkenEnBuurten_{year}.csv', index=False)

    def run(self): 
        for year in range(self.start_year, self.end_year): 
            self.identifier_dict[year] = self.get_identifier(year)
            self.download_data(year)
            print(f'cbs data for {year} downloaded to data/raw/cbs/wijkenEnBuurten_{year}.csv')

In [ ]:
cbs_data_downloader = CBSDataDownloader()


In [46]:
CBSDataDownloader(start_year = 2012, end_year = 2013).run()

HTTPError: Downloading table '70904ned' failed. 500 Server Error: Could not find a property named 'AantalInwoners_5' on type 'Cbs.OData.TData'. for url: https://opendata.cbs.nl/ODataFeed/odata/70904ned/TypedDataSet?%24format=json&%24select=Gemeentenaam_1%2CSoortRegio_2%2CCodering_3%2CAantalInwoners_5%2CHuishoudensTotaal_28%2CGemiddeldeHuishoudensgrootte_32%2CGemiddeldElektriciteitsverbruikTotaal_47%2CGemiddeldAardgasverbruikTotaal_55%2CPercentageWoningenMetStadsverwarming_63

In [37]:
# find table with wijken en buurten
wijken_buurten = toc[toc['Title'].str.contains('kerncijfers wijken en buurten', case=False)]
identifier_dict = {} 
for year in range(2012, 2024):
    try: 
        identifier_dict[year] = wijken_buurten[wijken_buurten.Title.str.contains(str(year))].Identifier.iloc[0]
    except: 
        print(f"no data found for {year}")

test_data = pd.DataFrame(cbsodata.get_data(identifier_dict[2019], select=[
                            'WijkenEnBuurten', 'Gemeentenaam_1', 'SoortRegio_2', 'Codering_3', 
                            'AantalInwoners_5', 'HuishoudensTotaal_28', 'GemiddeldeHuishoudensgrootte_32', 
                            'GemiddeldElektriciteitsverbruikTotaal_47', 'GemiddeldAardgasverbruikTotaal_55', 
                            'PercentageWoningenMetStadsverwarming_63'
                            ]))

test_data


,WijkenEnBuurten,Gemeentenaam_1,SoortRegio_2,Codering_3,AantalInwoners_5,HuishoudensTotaal_28,GemiddeldeHuishoudensgrootte_32,GemiddeldElektriciteitsverbruikTotaal_47,GemiddeldAardgasverbruikTotaal_55,PercentageWoningenMetStadsverwarming_63
0,Nederland,Nederland,Land,NL00,17282163,7924691,2.1,2730.0,1180.0,5.9
1,Aa en Hunze,Aa en Hunze,Gemeente,GM1680,25386,11152,2.2,3030.0,1740.0,NaN
2,Wijk 00 Annen,Aa en Hunze,Wijk,WK168000,3595,1560,2.3,2980.0,1620.0,NaN
3,Annen,Aa en Hunze,Buurt,BU16800000,3450,1490,2.3,2930.0,1590.0,NaN
4,Verspreide huizen Annen,Aa en Hunze,Buurt,BU16800009,150,65,2.3,4460.0,2340.0,NaN
...,...,...,...,...,...,...,...,...,...,...
17109,Gerenbroek,Zwolle,Buurt,BU01935150,3125,1205,2.6,2810.0,1110.0,NaN
17110,Oude Mars,Zwolle,Buurt,BU01935160,125,25,3.6,5130.0,1840.0,NaN
17111,Wijk 52 Soestweteringlanden,Zwolle,Wijk,WK019352,910,295,2.6,3580.0,1700.0,NaN
17112,Windesheim,Zwolle,Buurt,BU01935200,405,155,2.6,3090.0,1380.0,NaN
